In [82]:
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import seaborn as sns
from datetime import datetime
import random
import os
import optuna

In [83]:
def set_all_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

DEFAULT_RANDOM_SEED = 84
set_all_seeds(seed=DEFAULT_RANDOM_SEED)

In [105]:
path_big_train = '../train/data/train.csv'
path_big_test = '../train/data/test.csv'
df_train = pd.read_csv(path_big_train)
df_test = pd.read_csv(path_big_test)

In [85]:
import re

targets = ['target_month', 'target_day']
numerical = list(filter(lambda x: re.match(r'\d+[dm]_', x), df_train.columns))
categorical = ['kod_vrab', 'model',  'zavod_build', 'kuzov', 'telega', 'expected_srok_sl_y_b', 'date_build_b', 'manage_type', 'rod_id_x', 'reestr_state',
                'tormoz', 'tipvozd', 'tippogl', 'ownertype', 'is_in_kti']
dates = ['month', 'date_kap', 'date_dep', 'date_pl_rem', 'last_rem_date', 'date_build', 'srok_sl']

numerical += [
    "days_to_pl_rem",
    "days_from_last_rem",
    "ost_prob",
    "gruz",
    "cnsi_gruz_capacity",
    "cnsi_volumek",
    "tara",
    "cnsi_probeg_dr",
    "cnsi_probeg_kr",
    "norma_km",
    "expected_srok_sl_y"
]

numerical += ['kod_vrab_2_tr',
              'kod_vrab_3_tr', 'kod_vrab_5_tr', 'neis1_kod_tr_max',
              'neis2_kod_tr_max', 'neis3_kod_tr_max', 'gr_probeg_tr_max',
              'gr_probeg_tr_mean', 'gr_probeg_tr_min', 'por_probeg_tr_max',
              'por_probeg_tr_mean', 'por_probeg_tr_min', '_1m_tr_rems',
              'kod_vrab_0_pr', 'kod_vrab_1_pr', 'diff_road_pr', 'diff_st_pr',
              '_10d_pr_rems', '_20d_pr_rems', '_1m_pr_rems', '_allm_pr_rems',
              '_allm_tr_rems', 'last_month_for_cnt', 'months_after_tr_rems',
              'months_after_pr_rems']


notna_features = ['date_kap', 'date_dep', 'date_iskl']
notna_features_res = [feature + '_notna' for feature in notna_features]
categorical += notna_features_res

print('Unused columns:')
for col in df_train.columns:
    if col not in targets and col not in numerical and col not in categorical and col not in dates:
        print(col)

features = numerical + categorical + targets
train_features = numerical + categorical

medians = df_train[numerical].median()

def preprcoess_df(df, test=False):
    for date in dates:
        df[date] = pd.to_datetime(df[date])
    df = df[df['month'] > '2022-08-01']

    for col in notna_features:
        df[col + '_notna'] = df[col].notna().astype(int)

    features = numerical + categorical + targets
    if test:
        features = train_features
    df = df[features]

    df[categorical] = df[categorical].astype('str')
    df[numerical] = df[numerical].fillna(medians)

    return df

df_train = preprcoess_df(df_train)
df_test = preprcoess_df(df_test, test=True)

Unused columns:
wagnum
ownership_type
rod_id_y
date_iskl
days_to_iskl
days_to_srok_sl
iskl_in_a_year
ost_prob_in_a_month


In [106]:
df_train.shape, df_test.shape

((203610, 147), (33707, 147))

In [107]:
days_features = ['30d_next_time_prob', 'days_to_pl_rem', 'ost_prob_in_a_month', 'last_month_for_cnt', 'ost_prob_in_10_days', '_allm_pr_rems', 'days_since_dep', '5d_st_freigh_sign_mean', 'days_from_last_rem', '5d_avg_distance_min', 'ost_prob', '10d_nasip_mean', '5d_st_river_sign_mean', '5d_next_time_prob', '5d_diff_mean', '5d_nasip_mean', '30d_st_sea_sign_mean', '10d_st_sea_sign_mean', '10d_st_freigh_sign_mean', '10d_st_car_sign_mean']
cat_days_features = []
months_features = ['ost_prob_in_a_month', 'ost_prob_in_10_days', 'days_to_pl_rem', 'ost_prob', '30d_next_time_prob', '_allm_pr_rems', '5d_st_sea_sign_mean', 'gruz', '5d_st_car_sign_mean', '5d_st_river_sign_mean', '5d_st_freigh_sign_mean', 'days_to_srok_sl', '10d_st_car_sign_mean', '10d_st_sea_sign_mean', 'model', 'cnsi_gruz_capacity', '5d_opor_station_sign_mean', '10d_st_freigh_sign_mean', 'norma_km']
cat_months_features = ['season']

In [108]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier

def train(target, model_name, features, cat_features):
    cnt_folds = 5
    clfs = []
    scores = []
    kf = StratifiedKFold(n_splits=cnt_folds, shuffle=True, random_state=DEFAULT_RANDOM_SEED)

    #features = numerical + categorical + [target]
    features += cat_features + [target]
    df = df_train[features]

    if model_name == 'lgbm':
        for cat_feature in cat_features:
            df[cat_feature] = df[cat_feature].astype('category')
    else:
        for cat_feature in cat_features:
            le = LabelEncoder()
            df[cat_feature] = le.fit_transform(df[cat_feature])

    X = df.drop(target, axis=1)
    y = df[target]

    for ind, (train_index, test_index) in enumerate(kf.split(X=X, y=y)):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        if model_name == 'lgbm':
            clf = LGBMClassifier(random_state=DEFAULT_RANDOM_SEED)

            clf.fit(X_train, y_train,
                    eval_set=[(X_test, y_test)])
        else:
            clf = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=DEFAULT_RANDOM_SEED)

            clf.fit(X_train, y_train,
                    eval_set=[(X_test, y_test)],
                    verbose=150)

        clfs.append(clf)
        y_pred = clf.predict(X_test)
        score = f1_score(y_test, y_pred)
        scores.append(score)
        print(f'{ind} model: {score} f1')

    assert len(clfs) == cnt_folds

    print("mean f1 score --", np.mean(scores, dtype="float16"), np.std(scores).round(4))

    return clfs, np.mean(scores, dtype="float16"), np.std(scores).round(4)

In [ ]:
#param = {'iterations': 468, 'depth': 9, 'learning_rate': 0.20899008582405879,
#           'random_strength': 9, 'bagging_temperature': 0.9212431602262982,
#           'od_type': 'Iter', 'grow_policy': 'Depthwise'}

In [109]:
month_clfs_lgbm, month_f1_lgbm, month_std_lgbm = train('target_month', 'lgbm', months_features, cat_months_features)
day_clfs_lgbm, day_f1_lgbm, day_std_lgbm = train('target_day', 'lgbm', days_features, cat_days_features)
month_clfs_xg, month_f1_xg, month_std_xg = train('target_month', 'xg', months_features, cat_months_features)
day_clfs_xg, day_f1_xg, day_std_xg = train('target_day', 'xg', days_features, cat_days_features)

KeyError: "['ost_prob_in_10_days', '30d_next_time_prob', 'season'] not in index"

In [92]:
print(month_f1_lgbm, month_std_lgbm)
print(day_f1_lgbm, day_std_lgbm)
print(month_f1_xg, month_std_xg)
print(day_f1_xg, day_std_xg)

0.693 0.0062
0.575 0.0079
0.691 0.0051
0.5977 0.0224


In [94]:
(month_f1_lgbm + day_f1_lgbm + month_f1_xg + day_f1_xg) / 4

0.63916015625

In [102]:
def predict(clfs, df, name_model):
    features = numerical + categorical
    X = df[features]

    if name_model == 'lgbm':
        for cat_feature in categorical:
            X[cat_feature] = X[cat_feature].astype('category')
    else:
        for cat_feature in categorical:
            le = LabelEncoder()
            X[cat_feature] = le.fit_transform(X[cat_feature])

    y_pred = np.zeros((X.shape[0], 2))
    for clf in clfs:
        y_pred += clf.predict_proba(X)
    y_pred /= len(clfs)

    return y_pred

In [103]:
def make_predictions(clfs_day1, clfs_day2, clfs_month1, clfs_month2, df):
    pred_day1 = predict(clfs_day1, df, 'lgbm')
    pred_month1 = predict(clfs_month1, df, 'lgbm')
    pred_day2 = predict(clfs_day2, df, 'xg')
    pred_month2 = predict(clfs_month2, df, 'xg')

    pred_day = (pred_day1 + pred_day2) / 2
    pred_month = (pred_month1 + pred_month2) / 2
    pred_day = pred_day.argmax(axis=1)
    pred_month = pred_month.argmax(axis=1)

    df_test_new = df.copy()[['wagnum', 'month']]
    df_test_new['target_day'] = pred_day
    df_test_new['target_month'] = pred_month

    df_test_new = df_test_new[['wagnum', 'target_month', 'target_day']]
    sample = pd.read_csv('../train/target/y_predict.csv')
    sample = sample.merge(df_test_new, on='wagnum', how='left')

    return sample

prediction = make_predictions(day_clfs_lgbm, day_clfs_xg, month_clfs_lgbm, month_clfs_xg, df_test)
assert len(prediction) == len(df_test)

prediction.describe()

/tmp/ipykernel_159113/601968745.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_feature] = X[cat_feature].astype('category')
/tmp/ipykernel_159113/601968745.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_feature] = X[cat_feature].astype('category')
/tmp/ipykernel_159113/601968745.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

,wagnum,target_month,target_day
count,33707.000000,33707.000000,33707.000000
mean,17062.180437,0.042573,0.010502
std,9805.705493,0.201895,0.101943
min,0.000000,0.000000,0.000000
25%,8663.500000,0.000000,0.000000
50%,17115.000000,0.000000,0.000000
75%,25546.500000,0.000000,0.000000
max,33976.000000,1.000000,1.000000


In [97]:
prediction = make_predictions(day_clfs_xg, month_clfs_lgbm, df_test)
assert len(prediction) == len(df_test)

prediction.describe()

/tmp/ipykernel_159113/2886833591.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_feature] = X[cat_feature].astype('category')
/tmp/ipykernel_159113/2886833591.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[cat_feature] = X[cat_feature].astype('category')
/tmp/ipykernel_159113/2886833591.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

,wagnum,target_month,target_day
count,33707.000000,33707.000000,33707.000000
mean,17062.180437,0.041594,0.010829
std,9805.705493,0.199662,0.103497
min,0.000000,0.000000,0.000000
25%,8663.500000,0.000000,0.000000
50%,17115.000000,0.000000,0.000000
75%,25546.500000,0.000000,0.000000
max,33976.000000,1.000000,1.000000


In [104]:
prediction.to_csv('predictions/lgbm_xg_siam.csv', index=False)

In [14]:
cnt_folds = 3
df_features = sum(month_clfs[i].get_feature_importance(prettified=True).set_index('Feature Id') for i in range(cnt_folds)) / cnt_folds
df_features = df_features.sort_values('Importances', ascending=False)

In [28]:
df_features

,Importances
Feature Id,
ost_prob,14.070812
days_to_pl_rem,11.467984
5d_ost_prob_max,11.214305
5d_ost_prob_min,9.660306
10d_ost_prob_max,7.569389
...,...
diff_road_pr,0.000002
10d_skoroport_mean,0.000000
5d_skoroport_mean,0.000000


In [35]:
def opt_train(trial):
    target = 'target_month'

    param = {
        'iterations': trial.suggest_int('iterations', 100, 500),
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'random_strength': trial.suggest_int('random_strength', 0, 100),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])
    }

    clf = CatBoostClassifier(cat_features=categorical,
                             eval_metric='F1',
                             random_seed=DEFAULT_RANDOM_SEED,
                             allow_writing_files=False,
                             **param)

    features = numerical + categorical + [target]
    df = df_train[features]
    X = df.drop(target, axis=1)
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=DEFAULT_RANDOM_SEED)

    clf.fit(X_train, y_train,
            eval_set=(X_test, y_test),
            verbose=150, plot=False)

    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred)

    return score

In [36]:
study = optuna.create_study(direction='maximize')
study.optimize(opt_train, n_trials=100) # Можно изменить количество итераций

print("Лучшие параметры: ", study.best_params)

[I 2023-11-12 04:44:12,303] A new study created in memory with name: no-name-f68465eb-29f4-41c7-b256-ec96c2c15cc2


0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 231ms	remaining: 34.4s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.666553768
bestIteration = 82

Shrink model to first 83 iterations.


[I 2023-11-12 04:44:35,854] Trial 0 finished with value: 0.666553767993226 and parameters: {'iterations': 150, 'depth': 8, 'learning_rate': 0.25764691338078466, 'random_strength': 83, 'bagging_temperature': 0.39125301527748746, 'od_type': 'Iter', 'grow_policy': 'SymmetricTree'}. Best is trial 0 with value: 0.666553767993226.


0:	learn: 0.0000000	test: 0.0000000	best: 0.0000000 (0)	total: 105ms	remaining: 46.9s
150:	learn: 0.8324799	test: 0.6643461	best: 0.6685515 (96)	total: 16.5s	remaining: 32.4s
300:	learn: 0.9249509	test: 0.6589557	best: 0.6685515 (96)	total: 30.3s	remaining: 14.8s
447:	learn: 0.9688359	test: 0.6595148	best: 0.6685515 (96)	total: 53.9s	remaining: 0us

bestTest = 0.6685514718
bestIteration = 96

Shrink model to first 97 iterations.


[I 2023-11-12 04:45:31,975] Trial 1 finished with value: 0.668551471811076 and parameters: {'iterations': 448, 'depth': 9, 'learning_rate': 0.29990282741361146, 'random_strength': 43, 'bagging_temperature': 0.1946616268414929, 'od_type': 'IncToDec', 'grow_policy': 'Lossguide'}. Best is trial 1 with value: 0.668551471811076.


...

0:	learn: 0.3012312	test: 0.3123704	best: 0.3123704 (0)	total: 162ms	remaining: 22.3s
138:	learn: 0.8328969	test: 0.6683434	best: 0.6701255 (100)	total: 18.9s	remaining: 0us

bestTest = 0.670125523
bestIteration = 100

Shrink model to first 101 iterations.


[I 2023-11-12 05:28:12,294] Trial 98 finished with value: 0.6701255230125523 and parameters: {'iterations': 139, 'depth': 9, 'learning_rate': 0.1398394017826181, 'random_strength': 8, 'bagging_temperature': 0.9244377628815046, 'od_type': 'IncToDec', 'grow_policy': 'Depthwise'}. Best is trial 27 with value: 0.6761032472939217.


0:	learn: 0.2797647	test: 0.2930749	best: 0.2930749 (0)	total: 165ms	remaining: 27.5s
150:	learn: 0.8265967	test: 0.6672258	best: 0.6702395 (146)	total: 20.8s	remaining: 2.2s
166:	learn: 0.8537878	test: 0.6671125	best: 0.6702395 (146)	total: 23.4s	remaining: 0us

bestTest = 0.6702394909
bestIteration = 146

Shrink model to first 147 iterations.


[I 2023-11-12 05:28:37,620] Trial 99 finished with value: 0.6702394908725506 and parameters: {'iterations': 167, 'depth': 9, 'learning_rate': 0.12869587266109883, 'random_strength': 11, 'bagging_temperature': 0.9766346407415565, 'od_type': 'IncToDec', 'grow_policy': 'Depthwise'}. Best is trial 27 with value: 0.6761032472939217.


Лучшие параметры:  {'iterations': 468, 'depth': 9, 'learning_rate': 0.20899008582405879, 'random_strength': 9, 'bagging_temperature': 0.9212431602262982, 'od_type': 'Iter', 'grow_policy': 'Depthwise'}
